## Contents

1. [Fetching Economic Indicators](#fetching-economic-indicators)
2. [Feature Engineering](#feature-engineering)
3. [Fetching Stock Data](#fetching-stock-data)
4. [Selecting Initial Stocks](#selecting-initial_stocks)
5. [Initializing Models](#initializing-models)
6. [Stock Selection With Models](#stock-selection-with-models)
7. [Portfolio Optimization](#portfolio-optimization)

---

## Fetching Economic Indicators

In [15]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np

from pprint import pprint
from tqdm import tqdm

In [2]:
def fetch_economic_data(indicators, start, end, api_key=None):
    economic_data = {}
    for indicator in tqdm(indicators):
        try:
            data = web.DataReader(indicator, "fred", start, end, api_key=api_key)
            economic_data[indicator] = data
        except Exception as e:
            print(f"Could not fetch data for {indicator}. Error: {e}")
    return economic_data

In [4]:
import pickle

with open("data/base/economic_indicators.pkl", "rb") as f:
    economic_indicators_dict = pickle.load(f)

economic_indicators_dict

{'Gross Domestic Product': 'GDP',
 'Consumer Price Index for All Urban Consumers': 'CPIAUCNS',
 'Civilian Unemployment Rate': 'UNRATE',
 'Effective Federal Funds Rate': 'FEDFUNDS',
 'Trade Weighted U.S. Dollar Index': 'DTWEXM',
 '10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity': 'T10Y2Y',
 'M2 Money Stock': 'M2',
 'Personal Consumption Expenditures': 'PCE',
 'Homeownership Rate in the U.S.': 'HOANBS',
 'Total Business Inventories': 'BUSINV',
 'Industrial Production Index': 'INDPRO',
 'Real Personal Income Less Current Transfer Receipts': 'REALLN',
 'Real Disposable Personal Income': 'W875RX1',
 'Total Nonfarm Payrolls': 'PAYEMS',
 'Personal Saving Rate': 'PSAVERT',
 'Corporate Profits After Tax': 'A939RX0Q048SBEA',
 'National Financial Conditions Index': 'NFCI',
 'Velocity of M2 Money Stock': 'WM2NS',
 'University of Michigan: Consumer Sentiment': 'UMCSENT',
 'Crude Oil Prices': 'OILPRICE',
 'High Yield Corporate Bond Yield': 'BAMLH0A0HYM2',
 '10-Year Treasur

In [5]:
economic_indicators = [i for i in economic_indicators_dict.values()]
pprint(', '.join(economic_indicators))

('GDP, CPIAUCNS, UNRATE, FEDFUNDS, DTWEXM, T10Y2Y, M2, PCE, HOANBS, BUSINV, '
 'INDPRO, REALLN, W875RX1, PAYEMS, PSAVERT, A939RX0Q048SBEA, NFCI, WM2NS, '
 'UMCSENT, OILPRICE, BAMLH0A0HYM2, GS10, CP, STLFSI, USSLIND, GDPC1, USROA')


In [6]:
api_key = "7241f282cb7ac930cf729ca9d1acb3af"
economic_data = fetch_economic_data(economic_indicators, "1990-01-01", "2023-09-07", api_key=api_key)

100%|██████████| 27/27 [00:08<00:00,  3.13it/s]


In [7]:
freq = {k: pd.infer_freq(v.index) for k, v in economic_data.items()}
freq

{'GDP': 'QS-OCT',
 'CPIAUCNS': 'MS',
 'UNRATE': 'MS',
 'FEDFUNDS': 'MS',
 'DTWEXM': 'B',
 'T10Y2Y': 'B',
 'M2': 'W-MON',
 'PCE': 'MS',
 'HOANBS': 'QS-OCT',
 'BUSINV': 'MS',
 'INDPRO': 'MS',
 'REALLN': 'MS',
 'W875RX1': 'MS',
 'PAYEMS': 'MS',
 'PSAVERT': 'MS',
 'A939RX0Q048SBEA': 'QS-OCT',
 'NFCI': 'W-FRI',
 'WM2NS': 'W-MON',
 'UMCSENT': 'MS',
 'OILPRICE': 'MS',
 'BAMLH0A0HYM2': None,
 'GS10': 'MS',
 'CP': 'QS-OCT',
 'STLFSI': 'W-FRI',
 'USSLIND': 'MS',
 'GDPC1': 'QS-OCT',
 'USROA': 'QS-OCT'}

In [8]:
economic_data_resampled = {k: v.resample("W-FRI").mean() for k, v in economic_data.items()}
freq_resampled = {k: pd.infer_freq(v.index) for k, v in economic_data_resampled.items()}
freq_resampled

{'GDP': 'W-FRI',
 'CPIAUCNS': 'W-FRI',
 'UNRATE': 'W-FRI',
 'FEDFUNDS': 'W-FRI',
 'DTWEXM': 'W-FRI',
 'T10Y2Y': 'W-FRI',
 'M2': 'W-FRI',
 'PCE': 'W-FRI',
 'HOANBS': 'W-FRI',
 'BUSINV': 'W-FRI',
 'INDPRO': 'W-FRI',
 'REALLN': 'W-FRI',
 'W875RX1': 'W-FRI',
 'PAYEMS': 'W-FRI',
 'PSAVERT': 'W-FRI',
 'A939RX0Q048SBEA': 'W-FRI',
 'NFCI': 'W-FRI',
 'WM2NS': 'W-FRI',
 'UMCSENT': 'W-FRI',
 'OILPRICE': 'W-FRI',
 'BAMLH0A0HYM2': 'W-FRI',
 'GS10': 'W-FRI',
 'CP': 'W-FRI',
 'STLFSI': 'W-FRI',
 'USSLIND': 'W-FRI',
 'GDPC1': 'W-FRI',
 'USROA': 'W-FRI'}

In [9]:
economic_data_df = pd.concat([v for v in economic_data_resampled.values()], axis=1)
economic_data_df = economic_data_df.interpolate("linear")
economic_data_df

,GDP,CPIAUCNS,UNRATE,FEDFUNDS,DTWEXM,T10Y2Y,M2,PCE,HOANBS,BUSINV,...,WM2NS,UMCSENT,OILPRICE,BAMLH0A0HYM2,GS10,CP,STLFSI,USSLIND,GDPC1,USROA
DATE,,,,,,,,,,,,,,,,,,,,,
1990-01-05,5872.701000,127.400,5.400,8.2300,92.558300,0.067500,3161.1,3730.700,88.482000,NaN,...,3181.9,93.000,22.64100,NaN,8.210,268.869000,NaN,1.710,9364.259000,0.750000
1990-01-12,5879.418462,127.550,5.375,8.2325,91.898040,0.130000,3164.1,3730.075,88.452462,NaN,...,3195.7,92.125,22.50775,NaN,8.275,269.905385,NaN,1.645,9366.873154,0.745385
1990-01-19,5886.135923,127.700,5.350,8.2350,92.575125,0.082500,3164.4,3729.450,88.422923,NaN,...,3182.7,91.250,22.37450,NaN,8.340,270.941769,NaN,1.580,9369.487308,0.740769
1990-01-26,5892.853385,127.850,5.325,8.2375,92.790000,0.148000,3166.4,3728.825,88.393385,NaN,...,3163.6,90.375,22.24125,NaN,8.405,271.978154,NaN,1.515,9372.101462,0.736154
1990-02-02,5899.570846,128.000,5.300,8.2400,92.266640,0.166000,3167.6,3728.200,88.363846,NaN,...,3141.9,89.500,22.10800,NaN,8.470,273.014538,NaN,1.450,9374.715615,0.731538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-11,26798.605000,305.691,3.800,5.3300,90.981200,-0.728000,19370.8,18533.600,115.062000,2541097.0,...,20756.9,71.600,104.61000,3.914,4.170,2755.552000,-0.07,1.720,20386.467000,0.560000
2023-08-18,26798.605000,305.691,3.800,5.3300,90.981200,-0.694000,19370.8,18533.600,115.062000,2541097.0,...,20756.9,71.600,104.61000,3.884,4.170,2755.552000,-0.07,1.720,20386.467000,0.560000
2023-08-25,26798.605000,305.691,3.800,5.3300,90.981200,-0.720000,19370.8,18533.600,115.062000,2541097.0,...,20756.9,71.600,104.61000,3.926,4.170,2755.552000,-0.07,1.720,20386.467000,0.560000


In [10]:
nulls_pct = economic_data_df.isnull().sum() / len(economic_data_df)
nulls_pct[nulls_pct > 0]

BUSINV          0.059158
BAMLH0A0HYM2    0.207622
STLFSI          0.118316
dtype: float64

In [11]:
economic_data_df.dropna(inplace=True)
economic_data_df.to_csv("data/economic_data_weekly.csv")

## Feature Engineering

### The Fiscal Fortitude Quotient (FFQ)

The FFQ uses several economic indicators:
1. GDP
2. Unemployment Rate
3. Inflation
4. Federal Funds Rate
5. Oil Prices

Each is processed through its own piecewise function to capture complex relationships. Below are the functions and their explanations.

#### 1. GDP Component

$$
f(x) = 
\begin{cases}
    x & \text{if } x < 0.8 \\
    0.8 + 0.2(1 - (x - 0.8)^2) & \text{if } x \geq 0.8
\end{cases}
$$

In [12]:
def gdp_ffq(x):
    return x if x < 0.8 else 0.8 + 0.2 * (1 - (x - 0.8)**2)

**Rationale**: Moderate GDP growth is beneficial. However, extremely high GDP growth may indicate an overheated economy or a bubble. Hence, the function plateaus and then penalizes values above 0.8.

#### 2. Unemployment Rate Component

$$
f_{\text{UNRATE}}(x) = 
\begin{cases}
    x & \text{if } x > 0.2 \\
    0.2 - 0.2(1 - (x - 0.2)^2) & \text{if } x \leq 0.2
\end{cases}
$$

In [13]:
def unrate_ffq(x):
    return x if x > 0.2 else 0.2 - 0.2 * (1 - (x - 0.2)**2)

**Rationale**: Low unemployment is generally good, but extremely low levels may indicate labor shortages. Therefore, the function penalizes low values below 0.2.

#### 3. Inflation Component

$$
f_{\text{CPIAUCNS}}(x) = 1 - |x - 0.5| \times 2
$$

In [16]:
def inflation_ffq(x):
    return 1 - 2 * np.abs(x - 0.5)

**Rationale**: Moderate inflation is ideal. Both high inflation and deflation are unfavorable for economic health. The function is symmetrical around 0.5, penalizing both high and low values.